In [4]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle
import time

import data_utils
import model_utils
import train_utils
import evaluation
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)

import torch
import torch.nn as nn

### Run inference with Huggingface pretrained BERT fine-tuned on SQuAD

In [5]:
file_dict = {"train1.1": "./data/tiny_train-v1.1.json",
             "train2.0": "./data/tiny_train-v2.0.json", 
             "dev1.1": "./data/dev-v1.1.json"}
train_data_v1 = data_utils.prep_data(file_dict["train1.1"])
train_data_v2 = data_utils.prep_data(file_dict["train2.0"])
dev_data_v1 = data_utils.prep_data(file_dict["dev1.1"])

In [100]:
importlib.reload(model_utils)
importlib.reload(train_utils)
m = model_utils.PreTrainedSQuAD()
result = train_utils.inference(train_data_v2, m)

inference time (6 paragraphs): 57.1 sec
evaluation result:
 OrderedDict([('exact', 73.33333333333333), ('f1', 88.20374794287838), ('total', 45), ('HasAns_exact', 73.33333333333333), ('HasAns_f1', 88.20374794287838), ('HasAns_total', 45)])


### Run inference with Huggingface pretrained DistillBERT fine-tuned on SQuAD

In [7]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(train_data_v1, m)

inference time (6 paragraphs): 0.6 sec
exact match: 68.51851851851852, f1 score: 84.66444049777384


In [8]:
importlib.reload(model_utils)
m = model_utils.DistilBERTSQuAD()
result = train_utils.inference(dev_data_v1, m)

inference time (2067 paragraphs): 112.2 sec
exact match: 69.24314096499526, f1 score: 79.05754999708667
